1. Load the CSV file into a DataFrame.
2. Parse the "Radiomics" column, as it contains JSON data.
3. Remove columns with the same values across all rows.

In [12]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# create random seed for reproducibility
random_state = 42

# Load the data from DF_Radiomics_noduls_with_diagnose.csv
file_path = "DF_Radiomics_noduls_with_diagnose.csv"
data = pd.read_csv(file_path)

# Parse the JSON in the 'Radiomics' column
data['Radiomics'] = data['Radiomics'].apply(json.loads)

# Convert the 'Radiomics' column into separate columns
radiomics_data = pd.json_normalize(data['Radiomics'])

# Drop the original 'Radiomics' column
data = data.drop('Radiomics', axis=1)

# Combine the data with the new radiomics columns
data = pd.concat([data, radiomics_data], axis=1)

# Remove columns with the same value across all rows
data = data.loc[:, (data != data.iloc[0]).any()]

# Create a stratified split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=data['Labels'])

# Save the data to CSV files
train_data.to_csv('DF_Radiomics_noduls_with_diagnose_train_data.csv', index=False)
test_data.to_csv('DF_Radiomics_noduls_with_diagnose_test_data.csv', index=False)


In [19]:
data.head(2)

,Patient,Node,Labels,diagnostics_Image-original_Hash,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,...,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallDependenceEmphasis,original_gldm_SmallDependenceHighGrayLevelEmphasis,original_gldm_SmallDependenceLowGrayLevelEmphasis,diagnostics_Configuration_Settings.minimumROISize,diagnostics_Configuration_Settings.removeOutliers,diagnostics_Configuration_Settings.resampledPixelSpacing,diagnostics_Configuration_Settings.resegmentRange,diagnostics_Configuration_Settings.weightingNorm
0,LIDC-IDRI-0068,Node_N1,3.0,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,0506d1d0d6522eddd1640c8ea75c2fc5a9266270,...,0.053875,0.021012,0.488461,152.929922,0.019809,None,None,None,None,None
1,LIDC-IDRI-0068,Node_N1,3.0,bea2c9750ea59a0bebb6d3bd63ffacc40fcf6a28,"[0.683594, 0.683594, 1.25]","[512, 512, 261]",-1026.065264,-3024.0,3071.0,9d7da356d43e2f7ad7f374f6c193e97f6088d7c7,...,0.110650,0.024328,0.494688,165.356306,0.010062,None,None,None,None,None


In [15]:
print("Train data:", train_data.shape)
print("Test data:", test_data.shape)

Train data: (324, 108)
Test data: (81, 108)


# Fully Connected Neural Network

In [21]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Assuming 'data' is your pandas DataFrame
# Ensure the DataFrame only contains numeric values
data = data.apply(pd.to_numeric, errors='coerce')

# Split data into features and labels
X = data.drop('Labels', axis=1).values
y = data['Labels'].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X).float()
y_tensor = torch.tensor(y).float()

# Stratified split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, stratify=y_tensor)

# Create TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [29]:
print("Train data:", train_dataset.tensors[0].shape)

Train data: torch.Size([324, 107])


In [33]:
# Model
class FCNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FCNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.net(x)

# Hyperparameters
num_features = train_dataset.tensors[0].shape[1]  # Get the number of features from your dataset
input_size = 108  # Number of features
hidden_size = 64  # You can tune this
output_size = 4   # Assuming binary classification, adjust if necessary
learning_rate = 0.001
batch_size = 32
epochs = 1  # Adjust based on your runtime requirement

# Initialize model, loss function, and optimizer
model = FCNN(input_size, hidden_size, output_size).cuda()
criterion = nn.BCEWithLogitsLoss()  # For binary classification, change if needed
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)  # L2 regularization

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [31]:
#check if cuda is available, print the gpu model name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(torch.cuda.get_device_name(0))
    # Move model to the device
    model = model.to(device)

NVIDIA GeForce RTX 3090


In [34]:
# Training loop
model.fc = nn.Linear(num_features, output_size)  # 'num_output_neurons' is the number of output neurons in your linear layer

# Training loop
model.train()
for epoch in range(epochs):
    for inputs, targets in train_loader:
        # Move inputs and targets to the device
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x107 and 108x64)

In [ ]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs.squeeze(), y_test_tensor)
    print(f"Test Loss: {test_loss.item()}")